In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from src.arch import NN
from src import setup
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
from pathlib import Path
from scipy.signal import resample


In [6]:
#load data
raw_train = pd.read_csv(setup.raw_train_path) 
train_demographics = pd.read_csv(setup.train_demographics_path)
raw_test = pd.read_csv(setup.raw_test_path)
test_demographics = pd.read_csv(setup.test_demographics_path)

In [ ]:
# making 1D data lists for each sensor
gestures = raw_train['gesture'].unique()
subjects = raw_train['subject'].unique()

#sensor data lists
svm_data = []

#thermo data
thermo_data_1 = []
thermo_data_2 = []
thermo_data_3 = []
thermo_data_4 = []
thermo_data_5 = []

#tof data
tof_data_1 = []
tof_data_2 = []
tof_data_3 = []
tof_data_4 = []
tof_data_5 = []

gyro_data = []

#demographics data
shoulder_to_wrist_cm = []
elbow_to_wrist_cm = []
height = []
handedness = []
adult_child = []
sex = []
age = []

#labels for each gesture and subject
labels = []
for gesture in gestures:   
    for subject in subjects:
        subject_data = raw_train[(raw_train['gesture'] == gesture) & (raw_train['subject'] == subject)]
        subject_demographics = train_demographics[train_demographics['subject'] == subject]

        
        #calculating svm (sensor vector magnitude) from accelerometer data
        svm = np.sqrt(subject_data['acc_x']**2 + subject_data['acc_y']**2 + subject_data['acc_z']**2)
        svm_data.append(svm)
        
        #adding thermo data
        thermo_data_1.append(subject_data['thm_1'].values)
        thermo_data_2.append(subject_data['thm_2'].values)
        thermo_data_3.append(subject_data['thm_3'].values)
        thermo_data_4.append(subject_data['thm_4'].values)
        thermo_data_5.append(subject_data['thm_5'].values)

        #adding gyro data
        gyro_data.append(subject_data[['rot_w', 'rot_x', 'rot_y', 'rot_z']].values)

        #separating tof data into 8x8 grids, sequentially stacked (5 x 3D arrays, one per sensor)
        #tof_cols = [f'tof_{sensor}_v{pixel}' for sensor in range(1, 6) for pixel in range(64)]
        #tof_data.append(subject_data[tof_cols].values)

        labels.append(gesture)

        #adding demographics data
        shoulder_to_wrist_cm.append(subject_demographics['shoulder_to_wrist_cm'].values[0])
        elbow_to_wrist_cm.append(subject_demographics['elbow_to_wrist_cm'].values[0])
        height.append(subject_demographics['height_cm'].values[0])
        handedness.append(subject_demographics['handedness'].values[0])
        adult_child.append(subject_demographics['adult_child'].values[0])
        sex.append(subject_demographics['sex'].values[0])
        age.append(subject_demographics['age'].values[0])


In [63]:
svm_data_resampled = [resample(svm, 1000) for svm in svm_data] 

thermo_data_1_resampled = [resample(thermo, 1000) for thermo in thermo_data_1] 
thermo_data_2_resampled = [resample(thermo, 1000) for thermo in thermo_data_2]
thermo_data_3_resampled = [resample(thermo, 1000) for thermo in thermo_data_3]
thermo_data_4_resampled = [resample(thermo, 1000) for thermo in thermo_data_4]
thermo_data_5_resampled = [resample(thermo, 1000) for thermo in thermo_data_5]

#tof_data_resampled = [resample(tof, 1000) for tof in tof_data]      
gyro_data_resampled = [resample(gyro, 1000) for gyro in gyro_data] 



In [81]:
svm_data_array = np.array(svm_data_resampled)
thermo_data_1_array = np.array(thermo_data_1_resampled)
thermo_data_2_array = np.array(thermo_data_2_resampled)
thermo_data_3_array = np.array(thermo_data_3_resampled)
thermo_data_4_array = np.array(thermo_data_4_resampled)
thermo_data_5_array = np.array(thermo_data_5_resampled)

#tof_data_array = np.array(tof_data)
gyro_data_array = np.array(gyro_data_resampled)
print(svm_data_array.shape, gyro_data_array.shape, thermo_data_1_array.shape, thermo_data_2_array.shape, thermo_data_3_array.shape, thermo_data_4_array.shape, thermo_data_5_array.shape)

(1458, 1000) (1458, 1000, 4) (1458, 1000) (1458, 1000) (1458, 1000) (1458, 1000) (1458, 1000)


In [75]:
#splitting into train and val sets
input_data = np.concatenate((svm_data_array, thermo_data_1_array, thermo_data_2_array, thermo_data_3_array, thermo_data_4_array, thermo_data_5_array))

In [80]:
input_data.shape

(8748, 1000)